# Disaster Related News Classification
Using Machine Learning toold to categorize Twitter news into disaster related and not disaster related. Using labelled dataset to train the model. Using Naive Bayes model for classification.


In [ ]:
import nltk
import pandas as pd
import string
import random
from sklearn import metrics

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
data = pd.read_csv('/disaster_response_messages_training.csv').head(17000)

In [3]:
# Checking for inappropriate values
index = data[data['related'].isin([1, 0]) == False].index
data.drop(index=index, inplace=True)

In [4]:
# Taking only the columns we are looking at
messages = data.message.values
related = data.related.values

mid = int(len(messages) * 0.8)
messages_train = messages[:mid]
related_train = related[:mid]
messages_test = messages[mid:]
related_test = related[mid:]

In [5]:
def get_feature(message):
  message = message.lower()
  tokens = nltk.pos_tag(nltk.word_tokenize(message))
  tokens = [token[0] for token in tokens if (token[1] not in ['VBZ', 'DT', 'TO', 'EX', 'PDT', 'PRP', 'CD', 'PRP$'])]
  return tokens

In [6]:
# preparing dictionary
dictionary = []
for message in messages_train:
  dictionary.extend(get_feature(message))
random.shuffle(dictionary)

In [7]:
def grouping(message):
  return {word: (word in message) for word in dictionary}

In [8]:
# Extracting features
feature_set = []
for num, message in enumerate(messages_train):
  feature = grouping(nltk.word_tokenize(message.lower()))
  feature_set.append((feature, related_train[num]))

In [9]:
# Creating model
clf = nltk.NaiveBayesClassifier.train(feature_set)

In [11]:
# Prediction for the test bundle we have in our hand and finding the accuracy of the model for that
prediction = [clf.classify(grouping(nltk.word_tokenize(feature.lower()))) for feature in messages_test]
accuracy = metrics.accuracy_score(related_test, prediction)
print(accuracy)

0.8048708048708049


In [12]:
# Checking the messages which where predicted wrong
for num, message in enumerate(messages_test[:30]):
  prediction = clf.classify(grouping(nltk.word_tokenize(message.lower())))
  if prediction != related_test[num]:
    print(message)
    print(prediction)


The consultant will also arrange for and coordinate the fielding of regular monitoring and field assessment missions, as well as a final evaluation mission.
1
The earthquake in Pakistan will be the theme of an all-star fundraising gala, with pledges going to German Agro Action.
0


In [21]:
def classify_percentage(message):
  message = message.lower()
  sentence_feature = grouping(nltk.word_tokenize(message))
  probability = clf.prob_classify(sentence_feature)
  print('related:', probability.prob(1))
  print('unrelated:', probability.prob(0))


In [25]:
# Using the model to classify a sentence in percentage
classify_percentage('Earthquake in Japan')

related: 0.9518177823159734
unrelated: 0.048182217684024545
